all the imports

In [1]:
import arxiv
import requests
import json
from functools import lru_cache
import logging
import chromadb
from sentence_transformers import SentenceTransformer


c:\Users\Lenovo\.conda\envs\r-conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
@lru_cache(maxsize=100)


def get_arxiv_papers(query: str, max_papers: int = 5) -> list[dict]:
    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results = max_papers)
    papers = []

    for result in client.results(search):
        paper = {
            "title" : result.title,
            "authors": [a.name for a in result.authors],
            "summary": result.summary,
            "published": str(result.published.date()),
            "url": result.pdf_url
        }
        papers.append(paper)

    return papers

In [4]:
from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(5))


def get_ieee_papers(query:str, api_key:str, max_results:int = 5) -> list[dict]:
    url = "https://ieeexploreapi.ieee.org/api/v1/search/articles"
    params = {
        "querytext": query,
        'apikey': api_key,
        'max_records': max_results,
        'format': 'json'
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        papers = []
        
        for article in response.json().get('articles', []):
            papers.append({
                "title": article.get('title', 'no title'),
                "authors": [author["full_name"] for author in article.get('authors', [])],
                "abstract":article.get('abstract', 'No abstract'),
                'published': article.get('publication_date',""),
                'url': article.get('pdf_url', article.get('html_url',''))
            })
        return papers
    except Exception as e:
        raise Exception(f'IEEE API request failed: {str(e)}')

In [5]:
def research_agent(query:str, ieee_api_key:str, max_papers: int = 5)-> list[str]:
    try:
        papers = get_arxiv_papers(query, max_papers)
        if papers:
            return papers
    except Exception as e:
        print(f'arXiv failed: {str(e)}')
        
    #if ieee_api_key:
        #try:
            #return get_ieee_papers(query, ieee_api_key, max_papers)
        #except Exception as e:
            #print(f'IEEE failed: {str(e)}')
    #raise Exception("something's wrong with API connection")        



In [6]:
def get_results(papers: list[dict]):
    for i, paper in enumerate(papers,1):
        print(f"""
        [{i}] {paper['title']}
        Authors:{','.join(paper.get('authors', []))}
        Published: {paper.get('published', 'N/A')}
        Summary: {paper.get('summary', paper.get('abstract', 'N/A'))[:500]}...
        URL:{paper.get('url','N/A')}
        """)

In [7]:
papers = research_agent('LLM quantization', ieee_api_key="#")
get_results(papers)

INFO:arxiv:Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=LLM+quantization&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
INFO:arxiv:Got first page: 100 of 67384 total results



        [1] A Comprehensive Evaluation of Quantization Strategies for Large Language Models
        Authors:Renren Jin,Jiangcun Du,Wuwei Huang,Wei Liu,Jian Luan,Bin Wang,Deyi Xiong
        Published: 2024-02-26
        Summary: Increasing the number of parameters in large language models (LLMs) usually
improves performance in downstream tasks but raises compute and memory costs,
making deployment difficult in resource-limited settings. Quantization
techniques, which reduce the bits needed for model weights or activations with
minimal performance loss, have become popular due to the rise of LLMs. However,
most quantization studies use pre-trained LLMs, and the impact of quantization
on instruction-tuned LLMs and the rel...
        URL:http://arxiv.org/pdf/2402.16775v2
        

        [2] Achieving binary weight and activation for LLMs using Post-Training Quantization
        Authors:Siqing Song,Chuang Wang,Ruiqi Wang,Yi Yang,Xuyao Zhang
        Published: 2025-04-07
        Summary: 

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

client = chromadb.Client()
collection = client.create_collection('research_papers')

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
